In [ ]:
from grand.grandlib_classes.grandlib_classes import *
import grand.dataio.root_trees as rt
import numpy as np
import matplotlib.pyplot as plt
# import glob
import os
from scipy.fft import rfftfreq, rfft, irfft
from grand import ECEF, Geodetic, GRANDCS, LTP
from scipy.optimize import minimize_scalar
# from GP13_UD_T3_offline import grand_T3_trigger, safe_substraction, get_DU_coord
plt.style.use("/Users/xishui/Dropbox/Config/presentation.mplstyle")

In [ ]:
coord_1078 = Geodetic(latitude=40.99368437530295, longitude=93.95411072589444, height=1205.9284000000027)
coord_DAQ = Geodetic(latitude= 40.99734117, longitude=93.94868278, height=1205.9284000000027)

def get_DU_coord(lat, long, alt, obstime, origin=coord_DAQ):
  # From GPS to Cartisian coordinates
  geod = Geodetic(latitude=lat, longitude=long, height=alt)
  gcs = GRANDCS(geod, obstime=obstime, location=origin)
  return gcs

# Merge coinc table from different CD

In [ ]:
file_coinctable = np.zeros((0, 4), dtype=np.float64)
file_ducoord = np.zeros((0, 4), dtype=np.float64)
file_rec_sphere = np.zeros((0, 9), dtype=np.float64)
file_rec_plane = np.zeros((0, 8), dtype=np.float64)
file_du_id = np.zeros((0,1), dtype=np.int16)

i_event = 0
i_row = 0
for i_run in range(1,9):
    run_str = f"0914_UD_MD_11DUs_13h_Threshold200/{i_run:03d}"
    # Check the file size, skip the zeros
    if os.path.getsize(f"coincidence_table/{run_str}/Rec_coinctable.txt") == 0:
        continue
    _file_coinctable = np.genfromtxt(f"coincidence_table/{run_str}/Rec_coinctable.txt", dtype=float)
    _file_ducoord = np.genfromtxt(f"coincidence_table/{run_str}/coord_antennas.txt", dtype=float)
    _file_rec_sphere = np.genfromtxt(f"coincidence_table/{run_str}/Rec_sphere_wave_recons.txt", dtype=float, usecols=np.arange(9))
    _file_rec_plane = np.genfromtxt(f"coincidence_table/{run_str}/Rec_plane_wave_recons.txt", dtype=float)
    _file_du_id = np.genfromtxt(f"coincidence_table/{run_str}/DU_id.txt", usecols=1)
    # Update the event number and row number
    _n_row = len(_file_coinctable[:,0])
    _n_event = _file_coinctable[-1,1]
    _file_coinctable[:,0] = _file_coinctable[:,0] + i_row
    _file_coinctable[:,1] = _file_coinctable[:,1] + i_event
    _file_ducoord[:,0] = _file_ducoord[:,0] + i_row
    _file_rec_plane[:,0] = _file_rec_plane[:,0] + i_event
    _file_rec_sphere[:,0] = _file_rec_sphere[:,0] + i_event
    i_row += _n_row
    i_event += _n_event
    
    file_coinctable = np.append(_file_coinctable, file_coinctable, axis=0)
    file_ducoord = np.append(file_ducoord, _file_ducoord, axis=0)
    file_rec_sphere = np.append(file_rec_sphere, _file_rec_sphere, axis=0)
    file_rec_plane = np.append(file_rec_plane, _file_rec_plane, axis=0)
    file_du_id = np.append(file_du_id, _file_du_id)

# Reconstructed directions

In [ ]:
list_n_du = file_rec_plane[:,1]
list_chi2 = file_rec_plane[:,6]

zenith = file_rec_plane[:,2].copy() # Propagation driection of the em wave: from the source to the observer
zenith = 180 - zenith # From the observer to the source
zenith[zenith > 90] = 90 - (zenith[zenith > 90] - 90) # Reflect the up-going events to down-going
plt.hist(zenith, bins=np.linspace(0, 90, 451), label='Total', histtype='step', lw=3, color='k')
plt.hist(zenith[list_n_du == 6], bins=np.linspace(0, 90, 451), label='n$_{DU}=6$', histtype='step', lw=1)
plt.hist(zenith[list_n_du == 5], bins=np.linspace(0, 90, 451), label='n$_{DU}=5$', histtype='step', lw=1)

plt.legend()
plt.xlabel('Zenith [deg] (wrapped at the horizon)')
plt.grid()
# plt.xlim(65, 90)
# plt.xticks(np.arange(0, 91, 10))
plt.tight_layout()
plt.legend(loc='best')
# plt.savefig("imgs/zenith_dist_beacon_173.pdf");

In [ ]:
# change the coordinate system: "from source to observer" to "from observer to source"
# the conventional CR notation
azimuth = file_rec_plane[:,4] + 180
azimuth[azimuth > 360] = azimuth[azimuth > 360] - 360
plt.hist(azimuth, bins=np.linspace(0, 360, 7201), label='Total', histtype='step', lw=1, color='k', zorder=100)
plt.hist(azimuth[file_rec_plane[:,1] == 6], bins=np.linspace(0, 360, 7201), label='n$_{DU}=6$', histtype='step', lw=2)
plt.hist(azimuth[file_rec_plane[:,1] == 7], bins=np.linspace(0, 360, 7201), label='n$_{DU}=7$', histtype='step', lw=2)
plt.legend()
plt.xlabel("Azimuth [deg]")
# plt.xlim(27, 28)
plt.grid()
plt.tight_layout()
# plt.savefig("imgs/azimuth_dist_beacon_173.pdf")

In [ ]:
plt.figure(figsize=(12,4))
plt.plot(azimuth, zenith, marker='.', ls='', markersize=2)
plt.ylim(90, 0)

## (Real/expected) time delays for reconstructed events

## Time delay of PWF

In [ ]:
def time_delay_plane_wave(pars, t_exp, list_du_pos):
  theta_deg, phi_deg = pars
  theta, phi = np.radians(theta_deg, phi_deg) # Reconstructed direction of the source
  event_coord_shower_axis = np.dot(list_du_pos, [np.sin(theta) * np.cos(phi), np.sin(theta) * np.sin(phi), np.cos(theta)]) 
  time_delay_rec =  event_coord_shower_axis / 3e8 * 1e9 # in nanoseconds
  # use the first triggered DU as the time origin
  time_offset = time_delay_exp[mask_sort_time_delay][0] * 1e9 - (time_delay_rec[mask_sort_time_delay])
  chi2 = np.sum((time_delay_exp[mask_sort_time_delay] * 1e9 - (time_delay_rec[mask_sort_time_delay] + time_offset))**2) / (len(time_delay_exp) - 2)
  return chi2

In [ ]:
list_chi2_PWF = file_rec_plane[:,6]
# list_chi2 = np.zeros(len(file_rec_plane))
# for i, event_id in enumerate(file_rec_plane[:,0]):
#   list_linenumber = file_coinctable[file_coinctable[:,1] == event_id,0]
#   time_delay_exp = file_coinctable[file_coinctable[:,1] == event_id,2]
#   time_delay_exp_ns = time_delay_exp * 1e9 # trigger times of DUs in nanosecond
#   mask_sort_time_delay = np.argsort(time_delay_exp)
#   # Reconstructed direction of the source, "from source to observer notation"
#   rec_theta, rec_phi = np.radians(file_rec_plane[file_rec_plane[:,0] == event_id,[2,4]])
#   du_pos = file_ducoord[list_linenumber.astype("int"),1:]
#   event_coord_shower_axis = np.dot(du_pos, [np.sin(rec_theta) * np.cos(rec_phi), np.sin(rec_theta) * np.sin(rec_phi), np.cos(rec_theta)]) 
#   time_delay_rec =  event_coord_shower_axis / 3e8 * 1e9 # in nanoseconds
#   # use the first triggered DU as the time origin
#   t_exp = time_delay_exp_ns[mask_sort_time_delay] - time_delay_exp_ns[mask_sort_time_delay][0]
#   t_rec = time_delay_rec[mask_sort_time_delay] - time_delay_rec[mask_sort_time_delay][0]
#   chi2 = np.sum((t_exp - t_rec)**2) / (len(time_delay_exp) - 2)
#   list_chi2[i] = chi2
#   # plt.clf()  
#   # plt.plot(t_rec, t_exp, marker='.', ls='')
#   # # plt.axis("equal")
#   # plt.plot(np.array([0, 5000]), np.array([0, 5000]), ls='--', color='r')
#   # plt.xlabel(r"$\Delta t_{\rm exp} $ [ns]")
#   # plt.ylabel(r"$\Delta t_{\rm rec} $ [ns]")
#   # plt.title(f"Plane Wave Rec {event_id:.0f}")
#   # plt.grid(True)
#   # plt.tight_layout()
#   # plt.savefig(f"img/delay_plane_{event_id:.0f}.pdf")

## The distribution of $\chi^2$

In [ ]:
list_chi2_PWF = file_rec_plane[:,6] 
plt.hist(list_chi2, np.logspace(0, 5, 51), histtype='step', color='k')
plt.hist(list_chi2[file_rec_plane[:,1] == 6], np.logspace(-2, 6, 51), histtype='step')
plt.hist(list_chi2[file_rec_plane[:,1] == 7], np.logspace(-2, 6, 51), histtype='step')
# plt.hist(list_chi2_PWF, np.logspace(0, 5, 51), histtype='step', color='r')
plt.semilogy()
plt.ylabel("# of events")
plt.xlabel("$\chi^2$")
plt.grid()
plt.semilogx()

# Layout of GP13

In [ ]:
file_cood = np.genfromtxt("coincidence_table/test/coord_antennas.txt")
file_du_id = np.genfromtxt("coincidence_table/test/DU_id.txt", usecols=(1))

In [ ]:
# GPS provided by on-site measurement at GP13, independent measurements from ones on board
y = np.array([93.94177, 40.99434,  1262.3760,    1267,
93.94717,  40.99066,  1257.6813,    1263,
93.95256,  40.98818, 1256.2160,    1257,
93.95853,  40.98984, 1251.1682,    1250,
93.96260,  40.98816, 1248.0000,    1251,
93.94189,  40.98383, 1269.2376,    1273,
93.95225,  40.98456, 1265.5424,    1263,
93.96222,  40.98312, 1256.5299,    1261,
93.94732,  40.98151,  1268.0000,    1269,
93.95710,  40.98132, 1263.2480,    1266,
93.94690,  40.97682,  1274.5520,    1276,
93.95224,  40.97508, 1276.0000,    1279,
93.95789,  40.97689, 1272.6000,    1275,
93.94213,  40.98834,  1266.9760,    1268]).reshape((-1, 4))

In [ ]:
_coord_DAQ_GPS_onsite = Geodetic(latitude=40.99434, longitude=93.94177, height=1262.3760)
# coord_DAQ_GPS_onsite = GRANDCS(_coord_DAQ_GPS_onsite, obstime="2024-04-28", location=coord_1078)
coord_DAQ_GPS_onsite = GRANDCS(_coord_DAQ_GPS_onsite, obstime="2024-04-28",
                               location=Geodetic(latitude=y[1,1], longitude=y[1,0], height=y[1,2]))
# coord_DAQ_1076 = GRANDCS(coord_DAQ_GPS_onsite, obstime="2024-04-28", location=coord_1078)
coord_DAQ_1076 = GRANDCS(x=406.4, y=456.6, z=0, obstime="2024-04-28", location=coord_1078)
plt.figure(figsize=(6, 5))
# plt.subplot(121)
# plt.plot(coord_DAQ_1076.x[0], coord_DAQ_1076.y[0], marker='*', markersize=20)
# plt.plot(file_cood[:,1], file_cood[:,2], marker='.', ls='')
# plt.plot(file_cood[:,1], file_cood[:,2], marker='.', ls='')
# plt.grid()
# plt.axis("equal")
# plt.xlabel("x[m]")
# plt.ylabel("y[m]")
# plt.subplot(122)
plt.plot(-coord_DAQ_GPS_onsite.y[0], coord_DAQ_GPS_onsite.x[0], marker='+', markersize=20)
# plt.plot(-coord_DAQ_1076.y[0] + np.array([0, 1000 * np.sin(np.radians(26.8))]), coord_DAQ_1076.x[0] + 1000 * np.array([0, -np.cos(np.radians(26.8))]), ls='--')
# plt.plot(-coord_DAQ_1076.y[0], coord_DAQ_1076.x[0], marker='*', markersize=20)
plt.plot(-file_cood[:,2], file_cood[:,1], marker='.', ls='')
plt.plot(-file_cood[:,2], file_cood[:,1], marker='.', ls='')

plt.grid()
plt.axis("equal")
plt.xlabel("Easting[m]")
plt.ylabel("Northing[m]")
plt.tight_layout()
# plt.savefig("imgs/GP13_layout_7DUs.pdf")

# Trigger times relative to the first trigger for all events

In [ ]:
# From below, only select events with relatively small chi2
mask_chi2 = list_chi2_PWF > 1e-1
list_good_event_ids = file_rec_plane[mask_chi2,0].astype(int) # The linenumbers of good events
list_bad_event_ids = file_rec_plane[~mask_chi2,0].astype(int) # The linenumbers of good events
# Indices of the DUs/traces in the events with small chi2
list_good_DUs = np.array([i in list_good_event_ids for i in file_coinctable[:,1]])
list_du_unique = np.unique(file_du_id[list_good_DUs]).astype(int)

t_relative = np.zeros_like(file_coinctable[list_good_DUs,0], dtype=float)
n_event = np.sum(list_good_event_ids)
list_event_time = np.array([np.min(file_coinctable[file_coinctable[:,1] == i,2]) for i in list_good_event_ids]) 
list_event_time_bad = np.array([np.min(file_coinctable[file_coinctable[:,1] == i,2]) for i in list_bad_event_ids]) 

for i, row in enumerate(file_coinctable[list_good_DUs]):
    event_id = row[1]
    # Use the first triggered DU as the time origin
    t0 = np.min(file_coinctable[file_coinctable[:,1] == event_id,2])
    t_relative[i] = row[2] - t0

plt.figure(figsize=(14, 5))
for du in list_du_unique:
    t = t_relative[(file_du_id[list_good_DUs] == du)] * 1e9
    # print(du, np.unique(t - t_ref, return_counts=True))
    plt.hist(t, np.linspace(-0, 6000, 1000), histtype='step', label=du)
plt.legend(markerscale=10, ncol=6)
plt.xticks(np.arange(0, 6000, 400))
plt.semilogy()
# plt.yticks(np.arange(0, 3500, 238.4185791))
# plt.ylim(-50, 50)
plt.grid(True)
plt.xlabel("$t_{\\rm rel}$[ns]")
plt.ylabel("# of triggers");

## Event time of the first triggered DU

In [ ]:
plt.plot(list_event_time, list_good_event_ids, marker='.', ls='', markersize=1)

# print(rate_avg, "Hz")
# print(rate_avg / rate)
plt.ylabel("Event ID")
plt.xlabel("Event time[s]")
# plt.xlim(0, 10);

## Interval between two events

In [ ]:
plt.hist(np.diff(list_event_time), np.logspace(-4, 5, 51))
plt.loglog()
plt.xlabel("$\Delta t$[s]")
plt.grid(True);

## GPS positioning

In [ ]:
du_coord_dict = {}
for i in list_du_unique:
    du_coord_dict[i] = [np.unique(file_ducoord[file_du_id == i][:,1])[0],
                       np.unique(file_ducoord[file_du_id == i][:,2])[0],
                       np.unique(file_ducoord[file_du_id == i][:,3])[0]]

## GPS timing vs true arrival time

In [ ]:
list_time_delay_rec = np.zeros_like(file_coinctable[:,1])
list_time_delay_exp = np.zeros_like(file_coinctable[:,1])
list_time_delay_true = np.zeros_like(file_coinctable[:,1])
true_pos_source = [71, 38, 1312] # The averaged reconstructed source position from the SWF

for i, event_id in enumerate(file_rec_plane[:,0]):
  list_linenumber = (file_coinctable[file_coinctable[:,1] == event_id,0]).astype("int")
  time_delay_exp = file_coinctable[file_coinctable[:,1] == event_id,2]
  _list_du = file_du_id[list_linenumber]
  time_delay_exp_ns = time_delay_exp * 1e9 # ns
  # use the first triggered DU as the time origin
  time_0 = np.min(time_delay_exp_ns)
  arg_min = np.argmin(time_delay_exp_ns) # Pass the index of the min to the true delta T
  list_time_delay_exp[list_linenumber] = time_delay_exp_ns - time_0
  
  du_pos = file_ducoord[list_linenumber,1:]
  time_delay_true = np.sqrt(np.sum((du_pos - true_pos_source)**2, axis=1)) / 3e8 * 1e9
  time_0 = time_delay_true[arg_min]
  list_time_delay_true[list_linenumber] = time_delay_true - time_0

In [ ]:
for du in list_du_unique:
    plt.plot(list_time_delay_true[(file_du_id == du) & list_good_DUs], list_time_delay_exp[(file_du_id == du) & list_good_DUs], marker='.', ls='', markersize=8, label=du)

plt.plot([0, 7000], [0, 7000], c='k', ls='--')
plt.xlim(-100, 6000)
plt.ylim(-100, 6000)
plt.legend(markerscale=2, ncols=2, loc="best", frameon=False)
plt.xlabel("t_true[ns]")
plt.ylabel("t_exp (t_1013=0)[ns]")
plt.grid("True")
# plt.tight_layout()
# plt.axis("equal")
# plt.savefig("imgs/delay_true_exp_by_DU.pdf")

## Observed time vs True time

In [ ]:
list_delta_Ttrue = list_time_delay_exp - list_time_delay_true 
for du in list_du_unique:
    # mask_small = (np.abs(list_delta_Ttrue) < 1e4) 
    m = np.mean(list_delta_Ttrue[(file_du_id == du) & list_good_DUs])
    # print(du, np.std(list_delta_Ttrue[(file_du_id == du) & mask_small]), '\t', f"{m:.0f}",);
    plt.hist(list_delta_Ttrue[(file_du_id == du) & list_good_DUs] - m, np.linspace(-40.5, 40.5, 81), label=f"{du}", histtype='step', lw=2)
    # plt.hist(list_delta_T, np.logspace(0, 6), label='$\Delta t > 0$')
plt.xlabel("$\Delta t$[ns]")
plt.legend()
plt.ylim(0, 40)
# plt.loglog()
plt.grid(True)
# plt.savefig(f"imgs/hist_delta_t_{du}.pdf");

## Amplitude distribution

In [ ]:
for du in list_du_unique:
    x = plt.hist(file_coinctable[file_du_id == du, -1], histtype='step', label=du, bins=np.linspace(0, 400, 81))
    gcs = file_ducoord[file_du_id == du][0]
    d = np.sqrt((gcs[0] - true_pos_source[0])**2
                + (gcs[1] - true_pos_source[1])**2
                + (gcs[2] - true_pos_source[2])**2)
    plt.text(x[1][np.argmax(x[0])] - 10, np.max(x[0]), f"{du}:{d:.0f}m")
# plt.semilogy()
plt.grid(True)
plt.xlabel("Max of ADC")
plt.ylabel("# of Events")

# Multiplicity

In [ ]:
n_DU, n = np.unique(file_rec_plane[~mask_chi2,1], return_counts=True)
plt.bar(n_DU, n, width=0.7, alpha=0.4)
n_DU, n = np.unique(file_rec_plane[mask_chi2,1], return_counts=True)
plt.bar(n_DU, n, width=0.7, alpha=0.4)
plt.grid()
plt.xticks(np.arange(13))
plt.semilogy();

# Check the SWF

## Distribution of $\chi^2$

In [ ]:
list_chi2_SWF = file_rec_sphere[:,2]
plt.hist(list_chi2_SWF, np.logspace(-2, 4,), histtype='step')
plt.loglog()
# plt.legend()
plt.grid()
plt.xlabel(r"$\chi^2_{\rm SWF}$");

## The reconstructed source position

In [ ]:
source = np.array([0, 0, coord_DAQ.height[0]])

In [ ]:
x = file_rec_sphere[:,4]
y = file_rec_sphere[:,5]
z = file_rec_sphere[:,6]
plt.plot(x, y, marker='.', ls='', markersize=2)
# plt.plot(y[list_chi2 < 1e3], -x[list_chi2 < 1e3], marker='.', ls='')
# plt.plot(y[list_chi2 > 1e4], -x[list_chi2 > 1e4], marker='.', ls='')
# plt.xlim(-100, 200)
# plt.ylim(-200, 200)
plt.axis("equal")

In [ ]:
d = np.sqrt((x - source[0])**2 + (y - source[1])**2 + (z - source[2]))
plt.hist(d[list_chi2_PWF < 2e3], np.logspace(0, 5), histtype='step')
plt.hist(d[mask_chi2], np.logspace(0, 5), histtype='step')
plt.loglog()
plt.xlabel("Distance to (0,0, 1206)[m]");

In [ ]:
ax = plt.figure(figsize=(10, 8)).add_subplot(projection='3d')
ax.scatter(x[~mask_chi2], y[~mask_chi2], z[~mask_chi2], label='BKG events')
ax.scatter(x[mask_chi2], y[mask_chi2], z[mask_chi2], label='Beacon events')
ax.scatter(*source, marker='*', s=500)
ax.view_init(elev=30., azim=240, roll=0)
ax.set_xlabel("x[m]")
ax.set_ylabel("y[m]")
ax.set_zlabel("z[m]")
ax.legend()
plt.tight_layout()

## Reconstructed directions

In [ ]:
_theta = np.rad2deg(np.arccos( file_rec_sphere[:,6] / file_rec_sphere[:,8]))
_theta[_theta > 90] = 180 - _theta[_theta > 90]
_phi = np.rad2deg(np.arctan2(file_rec_sphere[:,4], file_rec_sphere[:,5]))

# plt.hist(_theta[list_chi2 < 1e3], np.linspace(0, 91, 901), histtype='step')
# plt.hist(_theta[list_chi2 > 1e4], np.linspace(0, 91, 901), histtype='step');
# plt.hist(_phi[~mask_chi2], np.linspace(0, 361, 3601), histtype='step')
# plt.hist(_phi[mask_chi2], np.linspace(0, 361, 3601), histtype='step');
plt.hist(_phi[~mask_chi2], 200, histtype='step')
plt.hist(_phi[mask_chi2], 200, histtype='step');
# plt.xlim(26, 28)

In [ ]:
# plt.plot(azimuth[list_chi2 < 1e3], marker='.', ls='')
# plt.plot(azimuth[list_chi2 > 1e4], marker='.', ls='')
plt.hist(_theta[~mask_chi2], np.linspace(0, 90, 91), histtype='step')
plt.hist(_theta[mask_chi2], np.linspace(0, 90, 91), histtype='step');

In [ ]:
plt.figure(figsize=(15, 4))
plt.hist(_phi[list_chi2 < 1e4], np.linspace(0, 360, 101), histtype='step')
plt.hist(_phi[list_chi2 > 1e4], np.linspace(0, 360, 101), histtype='step');
# plt.xlim(25, 35);

In [ ]:
(list_event_time[-1] - list_event_time[0])

In [ ]:
list_good_event_ids

In [ ]:
len(file_du_id)

In [ ]:
list_good_event_ids[:4]

## Frequency of DUs in the "good" events

In [ ]:
dus, ns = np.unique(file_du_id[list_good_DUs], return_counts=True)
mask_sort = np.argsort(ns)
print(dus[mask_sort])
print(ns[mask_sort] / np.sum(mask_chi2))

In [ ]:
list_du_ocurrence_40Hz = {}
for du, n in zip(dus, ns):
    list_du_ocurrence_40Hz[int(du)] = n / np.sum(mask_chi2)

In [ ]:
du = 1041
list_rate = [10, 20, 40, 100]
list_du_freq = [list_du_ocurrence_10Hz[du],
                list_du_ocurrence_20Hz[du],
                list_du_ocurrence_40Hz[du],
                list_du_ocurrence_100Hz[du]]
plt.plot(list_rate, list_du_freq, marker='.')
plt.xlabel("Rate [Hz]")
plt.ylabel("Occurence")

# GP13 CD check

In [ ]:
file_CD = rt.DataTree("GP13_CD_test.root")

# 0914 LAST RUN 20Hz Beacon

12 DUs (DU1018 excluded)  
DU1023 downed due to a burst up to ~1400Hz  
Notch filter: 132MHz, width=0.9


## Data

UD:  
GP13_20240914_113654_RUN0914_UD_20dB_12DUs_20Hz_75MHz_BeaconTest_001_dat.root  
GP13_20240914_113746_RUN0914_UD_20dB_12DUs_20Hz_75MHz_BeaconTest_002_dat.root

CD:  
GP13.RUN0914_CD_20dB_11DUs_20Hz_75MHz_BeaconTest.20240914193922.10000.1.dat 

# 0914 21h 1h UD run

threshold 